In [130]:
processing_dir = '/Users/homebase/Desktop/Python/Udemy Course/Milestone Projects/Daily-File-Organiser'

In [127]:
os.listdir(processing_dir)

['main.ipynb',
 'Archive',
 'Folder2',
 'Log',
 '.ipynb_checkpoints',
 'Folder1',
 'main.py',
 'Backup',
 'Downloads']

In [116]:
source = os.path.join(processing_dir)
destination = os.path.join(os.getcwd(), "Dest")

In [168]:
processing_dir = os.path.join(os.getcwd(), 'Processing')
destination = os.path.join(os.getcwd(), "Dest")
for folder, sub_folders, files in os.walk(processing_dir):
    
    if os.path.basename(folder) in ["Archive"]: #ignore Archive
            continue
    for file in files:
        if file.endswith(".pdf"):
            source = os.path.join(folder, file)
            shutil.copy(source, destination)
            time.sleep(2)
        if file.endswith(".csv"):
            source = os.path.join(folder, file)
            shutil.copy(source, destination)
            time.sleep(10)

processing_folders = os.path.join(processing_dir, "Archive")
for folder in os.listdir(processing_dir):
    source = os.path.join(processing_dir, folder)
    shutil.move(source, processing_folders)

In [271]:
#Inbound Process
import paramiko
import sys
import shutil
import time
import posixpath
import datetime

#Today's date
get_date = datetime.datetime.now().strftime("%Y%m%d_%H%M")

#path variables
local = os.path.join(os.getcwd(), 'Processing')
remote = '/Files/ScannedDocs/'
archive = os.path.join(local, "Archive")
 

#Paramiko instancw
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname = '192.168.64.3', username = 'testuser', password = 'TempPassword1')

with open(f"/Users/homebase/Desktop/Python/Udemy Course/Milestone Projects/OPUS/Processing/Log/Inbound/{get_date}", 'a') as f:
    f.write(f"============= {get_date} Processing =============\n")
    with client.open_sftp() as sftp:
        for folder, sub_folders, files in os.walk(local):
            if os.path.basename(folder) in ['Archive', 'Log', 'Outbound']:
                continue
    
            for file in files:
                if file.endswith(".pdf") or file.endswith(".csv"):
                    local_source = os.path.join(folder, file)
                    remote_destination = posixpath.join(remote, file)
                    sftp.put(local_source, remote_destination)
                    f.write(f"Uploading {os.path.basename(local_source)} to {remote}\n")
                    
            if file.endswith(".pdf"):
                time.sleep(2)
    
            if file.endswith(".csv"):
                time.sleep(10)
    client.close()
    
    
    for folder in os.listdir(local):
        if os.path.basename(folder) in ['Archive', 'Log', '.ipynb_checkpoints', 'Outbound']:
                continue
        source = os.path.join(local, folder)
        shutil.move(source, archive)
        f.write(f"{os.path.basename(folder)} moved into Archive \n")

In [166]:
source = os.path.join(os.getcwd(), 'Processing')

In [167]:
os.listdir(source)

['Archive', 'Folder2', 'Folder1']

In [178]:
import datetime
get_date = datetime.datetime.now().strftime("%Y%m%d_%H%M")

In [179]:
get_date

'20251005_1621'

In [257]:
import os
import shutil

destination = os.path.join(os.getcwd(), 'Processing/Outbound')
outbound_file = os.path.join(os.getcwd(), "Outbound Folder")
for file in os.listdir(outbound_file):
    source_file = os.path.join(outbound_file, file)
    
    if os.path.isfile(source_file): #skip folders
        if file.endswith(".pdf"):
            shutil.move(source_file, destination)
            print(f"{os.path.basename(source_file)} moved to {os.path.basename(destination)}")

file1.pdf moved to Outbound
file2.pdf moved to Outbound
file3.pdf moved to Outbound
file4.pdf moved to Outbound
file5.pdf moved to Outbound


In [220]:
outbound_file

'/Users/homebase/Desktop/Python/Udemy Course/Milestone Projects/OPUS/Outbound Folder'

In [251]:
destination = os.path.join(os.getcwd(), 'Processing/Outbound')

In [252]:
destination

'/Users/homebase/Desktop/Python/Udemy Course/Milestone Projects/OPUS/Processing/Outbound'

In [222]:
for file in os.listdir(outbound_file):
    print(file)

file1.pdf
folder2
file2.pdf
file3.pdf
file4.pdf
file5.pdf
.ipynb_checkpoints
folder1


In [277]:
#Otbound Process

#path variables
local_path = os.path.join(os.getcwd(), 'Processing/Outbound/Reprocess')
remote = '/Files/OPUS/'

#Paramiko instance
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname = '192.168.64.3', username = 'testuser', password = 'TempPassword1')

with open(f"Processing/Log/Outbound/{get_date}_outbound_log", 'a') as f:
    f.write(f"============= {get_date} Outbound Processing =============\n")
    with client.open_sftp() as sftp:
        remote_files = sftp.listdir(remote) #os.listdir()
        for file in remote_files:
            if file.endswith(".pdf"):
                source = posixpath.join(remote, file)
                download_path = os.path.join(local_path, file)
                sftp.get(source, download_path)
                f.write(f"{os.path.basename(source)} downloaded to {os.path.basename(local_path)}\n")

                sftp.remove(source)
                f.write(f"{os.path.basename(source)} deleted\n")
    client.close()